In [1]:
import numpy as np
import pandas as pd
import csv
import sys
import os
  
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
import warnings
import pickle

from sklearn.ensemble import RandomForestClassifier

max_iters = 501
#n_estimators = 10

def rfc(x,y,filename):

   # Model output file name
   file = (os.path.splitext(filename))[0]
   fname = './models/rfc_' + file +'/'

   # File for writing precision,recall, f-measure scores for fraud transactions
   f = open('./prf/rfc_'+ file + '_prf' +'.txt' ,'w')
   f.write('precision,recall,f-score \n')

   # Stratified sampling based on Y
   X_train, X_test, y_train, y_test = train_test_split(x, y,stratify=y , test_size=0.30, random_state=42)

   # Create 15% validation set and 15% test set split
   X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,stratify=y_test , test_size=0.50, random_state=42)
   
   #Iterations
   it = 1
   
   # Run training algorithm for multiple class weights
   while it < max_iters:
       cw = {}
       cw[0] = 1
       cw[1] = it
       # Train
       print('**************************************')
       print("Iteration number  " , it)
       rfc = RandomForestClassifier(n_jobs=2, random_state=0, class_weight = cw)
       #lr = LogisticRegression(class_weight = cw)
       print('Class weights ', cw)
       rfc.fit(X_train,y_train)

       # Save trained model to disk
       name = fname + str(cw[1]) + '.sav'
       pickle.dump(rfc, open(name, 'wb'))

       # Predict on validation data
       y_val_pred = rfc.predict(X_val)
       print('Performance on validation data - Confusion matrix')
       print(confusion_matrix(y_val,y_val_pred))
   
       precision,recall,fscore,support=score(y_val,y_val_pred,average=None)
       print('Precision, Recall, F-score, Support  on validation data' )
       print("Precision" , precision)
       print("Recall" , recall)
       print("F-score" , fscore)
       print("Support" , support)

       p1 = precision[1]
       r1 = recall[1]
       f1 = fscore[1]

       f.write(str(p1) +','+ str(r1) + ',' + str(f1) + '\n') 
       it += 1

   f.close()

def run():
   filename = 'full_data.csv'
   df = pd.read_csv(filename, usecols = [8,9,10,11,12,13,14] , header = 0)
   
   results = list(map(int, df['Anomaly'])) 
   print('Number of fraudulent transactions ' , sum(results))

   features = ['Transaction_Type', 'Airfare', 'Lodging', 'Meals', 'Other_Transportation', 'Other_Expenses']
   targets = ['Anomaly']

   # Separating out the features and target variables
   X = df.loc[:, features].values
   y = df.loc[:, targets].values

    
   from sklearn.preprocessing import LabelEncoder, OneHotEncoder
   labelencoder_X = LabelEncoder()
   X[:,0] = labelencoder_X.fit_transform(X[:,0])
   onehotencoder = OneHotEncoder(categorical_features = [0])
   x = onehotencoder.fit_transform(X).toarray() 

   scaler = preprocessing.StandardScaler()
   x = scaler.fit_transform(x) 
    
   y  = [i for j in y for i in j]
   
   #Ignore warnings
   warnings.filterwarnings("ignore", category=FutureWarning)

   print("***********Logistic Regression**********")
   rfc(x,y,filename)
  
run()

Number of fraudulent transactions  8213


/home/sandisk/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/sandisk/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


***********Logistic Regression**********
**************************************
Iteration number   1
Class weights  {0: 1, 1: 1}
Performance on validation data - Confusion matrix
[[953125     36]
 [   280    952]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970632 0.96356275]
Recall [0.99996223 0.77272727]
F-score [0.99983426 0.85765766]
Support [953161   1232]
**************************************
Iteration number   2
Class weights  {0: 1, 1: 2}
Performance on validation data - Confusion matrix
[[953127     34]
 [   270    962]]
Precision, Recall, F-score, Support  on validation data
Precision [0.9997168  0.96586345]
Recall [0.99996433 0.78084416]
F-score [0.99984055 0.86355476]
Support [953161   1232]
**************************************
Iteration number   3
Class weights  {0: 1, 1: 3}
Performance on validation data - Confusion matrix
[[953127     34]
 [   275    957]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99971156 0.965691

Performance on validation data - Confusion matrix
[[953117     44]
 [   279    953]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970736 0.9558676 ]
Recall [0.99995384 0.77353896]
F-score [0.99983058 0.85509197]
Support [953161   1232]
**************************************
Iteration number   25
Class weights  {0: 1, 1: 25}
Performance on validation data - Confusion matrix
[[953119     42]
 [   276    956]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99971051 0.95791583]
Recall [0.99995594 0.77597403]
F-score [0.99983321 0.8573991 ]
Support [953161   1232]
**************************************
Iteration number   26
Class weights  {0: 1, 1: 26}
Performance on validation data - Confusion matrix
[[953120     41]
 [   280    952]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970631 0.95871098]
Recall [0.99995699 0.77272727]
F-score [0.99983163 0.85573034]
Support [953161   1232]
**********************************

Performance on validation data - Confusion matrix
[[953127     34]
 [   282    950]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970422 0.96544715]
Recall [0.99996433 0.7711039 ]
F-score [0.99983426 0.85740072]
Support [953161   1232]
**************************************
Iteration number   49
Class weights  {0: 1, 1: 49}
Performance on validation data - Confusion matrix
[[953122     39]
 [   275    957]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99971156 0.96084337]
Recall [0.99995908 0.77678571]
F-score [0.99983531 0.85906643]
Support [953161   1232]
**************************************
Iteration number   50
Class weights  {0: 1, 1: 50}
Performance on validation data - Confusion matrix
[[953129     32]
 [   280    952]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970632 0.96747967]
Recall [0.99996643 0.77272727]
F-score [0.99983636 0.85920578]
Support [953161   1232]
**********************************

...

Precision, Recall, F-score, Support  on validation data
Precision [0.99970002 0.96040609]
Recall [0.99995908 0.76785714]
F-score [0.99982954 0.8534055 ]
Support [953161   1232]
**************************************
Iteration number   402
Class weights  {0: 1, 1: 402}
Performance on validation data - Confusion matrix
[[953125     36]
 [   283    949]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970317 0.96345178]
Recall [0.99996223 0.77029221]
F-score [0.99983268 0.85611186]
Support [953161   1232]
**************************************
Iteration number   403
Class weights  {0: 1, 1: 403}
Performance on validation data - Confusion matrix
[[953125     36]
 [   283    949]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970317 0.96345178]
Recall [0.99996223 0.77029221]
F-score [0.99983268 0.85611186]
Support [953161   1232]
**************************************
Iteration number   404
Class weights  {0: 1, 1: 404}
Performance on validati

Performance on validation data - Confusion matrix
[[953124     37]
 [   284    948]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970212 0.96243655]
Recall [0.99996118 0.76948052]
F-score [0.99983163 0.85520974]
Support [953161   1232]
**************************************
Iteration number   426
Class weights  {0: 1, 1: 426}
Performance on validation data - Confusion matrix
[[953125     36]
 [   285    947]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970107 0.96337742]
Recall [0.99996223 0.76866883]
F-score [0.99983163 0.85507901]
Support [953161   1232]
**************************************
Iteration number   427
Class weights  {0: 1, 1: 427}
Performance on validation data - Confusion matrix
[[953126     35]
 [   284    948]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970212 0.96439471]
Recall [0.99996328 0.76948052]
F-score [0.99983268 0.85598194]
Support [953161   1232]
******************************

Precision, Recall, F-score, Support  on validation data
Precision [0.99970946 0.96954315]
Recall [0.99996853 0.77516234]
F-score [0.99983898 0.86152458]
Support [953161   1232]
**************************************
Iteration number   449
Class weights  {0: 1, 1: 449}
Performance on validation data - Confusion matrix
[[953127     34]
 [   271    961]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99971575 0.96582915]
Recall [0.99996433 0.78003247]
F-score [0.99984003 0.86304445]
Support [953161   1232]
**************************************
Iteration number   450
Class weights  {0: 1, 1: 450}
Performance on validation data - Confusion matrix
[[953126     35]
 [   273    959]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99971366 0.96478873]
Recall [0.99996328 0.77840909]
F-score [0.99983845 0.86163522]
Support [953161   1232]
**************************************
Iteration number   451
Class weights  {0: 1, 1: 451}
Performance on validati

Performance on validation data - Confusion matrix
[[953127     34]
 [   276    956]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99971051 0.96565657]
Recall [0.99996433 0.77597403]
F-score [0.9998374  0.86048605]
Support [953161   1232]
**************************************
Iteration number   473
Class weights  {0: 1, 1: 473}
Performance on validation data - Confusion matrix
[[953129     32]
 [   276    956]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99971051 0.96761134]
Recall [0.99996643 0.77597403]
F-score [0.99983845 0.86126126]
Support [953161   1232]
**************************************
Iteration number   474
Class weights  {0: 1, 1: 474}
Performance on validation data - Confusion matrix
[[953128     33]
 [   276    956]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99971051 0.96663296]
Recall [0.99996538 0.77597403]
F-score [0.99983793 0.86087348]
Support [953161   1232]
******************************

Precision, Recall, F-score, Support  on validation data
Precision [0.99970422 0.96544715]
Recall [0.99996433 0.7711039 ]
F-score [0.99983426 0.85740072]
Support [953161   1232]
**************************************
Iteration number   496
Class weights  {0: 1, 1: 496}
Performance on validation data - Confusion matrix
[[953128     33]
 [   281    951]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970527 0.96646341]
Recall [0.99996538 0.77191558]
F-score [0.99983531 0.85830325]
Support [953161   1232]
**************************************
Iteration number   497
Class weights  {0: 1, 1: 497}
Performance on validation data - Confusion matrix
[[953132     29]
 [   278    954]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99970842 0.97049847]
Recall [0.99996957 0.77435065]
F-score [0.99983898 0.86139955]
Support [953161   1232]
**************************************
Iteration number   498
Class weights  {0: 1, 1: 498}
Performance on validati